In [31]:
# imports
from csv import DictReader, DictWriter
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [32]:
og_column_names = ['station_id', 'longitude', 'latitude', 'time', 'AtmospherePressure', 'WindDirection', 'WindSpeed', 'Gust', 'WaveHeight', 'WavePeriod', 'MeanWaveDirection', 'Hmax', 'AirTemperature', 'DewPoint', 'SeaTemperature', 'RelativeHumidity']

column_names = ['AtmospherePressure', 'WindDirection', 'WindSpeed', 'Gust', 'AirTemperature', 'SeaTemperature', 'RelativeHumidity', 'WaveHeight', 'WavePeriod']

with open('../data/raw_data.csv', 'r') as f1, open('../data/clean_data.csv', 'w') as f2:
    reader = DictReader(f1, fieldnames=og_column_names)
    writer = DictWriter(f2, fieldnames=column_names, lineterminator='\n')

    next(reader)
    next(reader)

    writer.writeheader()

    for line in tqdm(reader):
        if any(line[column_name] == 'NaN' for column_name in column_names):
            continue
        writer.writerow({column_name: line[column_name] for column_name in column_names})


613392it [00:02, 264630.82it/s]


In [33]:
df = pd.read_csv('../data/clean_data.csv')
df.describe()

,AtmospherePressure,WindDirection,WindSpeed,Gust,AirTemperature,SeaTemperature,RelativeHumidity,WaveHeight,WavePeriod
count,385474.000000,385474.000000,385474.000000,385474.000000,385474.000000,385474.000000,385474.000000,385474.000000,385474.000000
mean,1013.791675,206.580169,15.119516,21.455321,11.606788,12.435129,82.288061,2.319344,6.144468
std,12.241422,89.416027,6.891260,9.537482,2.879286,2.299281,9.718917,1.517461,1.728072
min,910.300000,0.000000,0.000000,0.000000,0.264000,0.200000,0.391000,0.100000,2.000000
25%,1006.400000,150.000000,10.000000,14.800000,9.500000,10.700000,75.000000,1.200000,5.000000
50%,1015.039000,220.000000,15.000000,20.000000,11.600000,12.271000,83.000000,2.000000,6.000000
75%,1022.400000,270.000000,19.923000,27.000000,14.000000,14.300000,90.000000,3.000000,7.000000
max,1046.400000,360.000000,76.506000,200.000000,22.104000,24.200000,100.000000,14.100000,18.000000


In [34]:
# Use the interquartile range to remove the outliers in data.
for column_name in column_names:

    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1

    lower_whisker = Q1 - 1.5*IQR
    upper_whisker = Q3 + 1.5*IQR
    df = df[(df[column_name] >= lower_whisker) & (df[column_name] <= upper_whisker)]

df.describe()

,AtmospherePressure,WindDirection,WindSpeed,Gust,AirTemperature,SeaTemperature,RelativeHumidity,WaveHeight,WavePeriod
count,361891.000000,361891.000000,361891.000000,361891.000000,361891.000000,361891.000000,361891.000000,361891.000000,361891.000000
mean,1014.725728,204.739858,14.541314,20.516864,11.710947,12.498833,82.573291,2.102670,5.959147
std,11.323496,90.193495,6.320948,8.476560,2.878474,2.329885,9.596186,1.185541,1.548250
min,982.400000,0.000000,0.000000,0.000000,2.964000,5.900000,52.734000,0.100000,2.000000
25%,1007.400000,150.000000,10.000000,14.000000,9.600000,10.700000,76.000000,1.200000,5.000000
50%,1015.600000,217.000000,14.010000,20.000000,11.748000,12.400000,83.203000,1.900000,6.000000
75%,1022.656000,270.000000,19.000000,25.957000,14.100000,14.400000,90.000000,2.800000,7.000000
max,1046.400000,360.000000,33.300000,45.824000,20.600000,19.200000,100.000000,5.600000,10.000000


In [35]:
features = column_names[:-2]
labels = column_names[-2:]

X = df[features].to_numpy()
# Note that we have 2 labels
Y = df[labels].to_numpy()

In [36]:
# The commonly used split of 75% for training, 10% for validation, and 15%
# for the test sets has been followed when splitting the dataset.
# V Roshan Joseph. Optimal ratio for data splitting. Statistical Analysis and Data Mining: The
# ASA Data Science Journal, 15(4):531–538, 2022.

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.6, random_state=42)

# y1: WaveHeight
y1_train = Y_train[:,0]
y1_val = Y_val[:,0]
y1_test = Y_test[:,0]

# y2: WavePeriod
y2_train = Y_train[:,1]
y2_val = Y_val[:,1]
y2_test = Y_test[:,1]

In [37]:
def finding_polynomial(X_train, X_val, y_train, y_val, highest_degree=3):
    # define a list of values for polynomial degrees
    degrees = [i for i in range(1, highest_degree + 1)]
    # declare a variable to store the resulting validation errors for each polynomial degree
    val_errors = []
    for i in range(len(degrees)):    # use for-loop to fit polynomial regression models with different degrees

        poly = PolynomialFeatures(degree=degrees[i])
        X_train_poly = poly.fit_transform(X_train)
        X_val_poly = poly.fit_transform(X_val)

        regression = LinearRegression()
        regression.fit(X_train_poly, y_train)

        y_pred = regression.predict(X_val_poly)

        val_error = mean_squared_error(y_val, y_pred)

        val_errors.append(val_error)

    return val_errors

In [38]:
val_errors_1 = finding_polynomial(X_train, X_val, y1_train, y1_val)
val_errors_2 = finding_polynomial(X_train, X_val, y2_train, y2_val)
print(val_errors_1)
print(val_errors_2)

[0.7968269127042754, 0.723473876764308, 0.6757964510629035]
[2.087114624171925, 1.8891810928240123, 1.7793190205935225]
